In [ ]:
# download the weaviate client
%pip install -U weaviate-client

In [ ]:
import weaviate
import weaviate.classes as wvc
from weaviate.collections import Collection
from weaviate.client import WeaviateClient

client_src = weaviate.connect_to_weaviate_cloud(
		cluster_url="<SOURCE_CLUSTER_URL>",
		auth_credentials=weaviate.auth.AuthApiKey("<KEY>"))

client_tgt = weaviate.connect_to_weaviate_cloud(
		cluster_url="<TARGET_CLUSTER_URL>",
		auth_credentials=weaviate.auth.AuthApiKey("<KEY>"),
        headers={"X-OpenAI-Api-Key":"<KEY>"})

def create_collection(client_in: WeaviateClient, collection_name: str, enable_mt=False):

    reviews = client_in.collections.create(
        name=collection_name,
        multi_tenancy_config=wvc.config.Configure.multi_tenancy(enabled=enable_mt),
    )

    return reviews

reviews_tgt = create_collection(client_tgt, "papers", enable_mt=False)


In [ ]:
from tqdm import tqdm

def migrate_data(collection_src: Collection, collection_tgt: Collection):

    with collection_tgt.batch.fixed_size(batch_size=100) as batch:
        for q in tqdm(collection_src.iterator(include_vector=True)):
            batch.add_object(
                properties=q.properties,
                vector=q.vector["default"],
                uuid=q.uuid
            )
            # Log the insertion of each object
            print(f"Object with UUID {q.uuid} added.")
    return True


reviews_src = client_src.collections.get("papers")
reviews_tgt = client_tgt.collections.get("papers")

migrate_data(reviews_src, reviews_tgt)

client_src.close()
client_tgt.close()